<a href="https://colab.research.google.com/github/andresmg42/Proyectos-ADAII/blob/main/Copy_3_of_proyect1_brute_force.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECT 1(DINAMIC AND GREEDY PROGRAMING)

In [1]:
# structure data definition

#inputs:

#M: list of tuples (Mi,mi)

#E: list of tuples of (ej,msj)

class Subject:
  def __init__(self,code,quotas):
    self.code:str=code
    self.quotas:int=quotas

class Student:
  def __init__(self,code,solicited_subjects,insatisfaction=0):
    self.code:str=code
    self.solicited_subjects:List[SolicitedSubject]=solicited_subjects
    self.assigned_subjects:List[SolicitedSubject]=[]


  def return_info(self):
    string=f'\n{self.code}\n'
    for s_subject in self.solicited_subjects:
       string+=s_subject.return_info()
    return string

  def return_info_sol(self):
    string=f'\n{self.code}\n'
    for s_subject in self.assigned_subjects:
       string+=s_subject.return_info()
    return string


  def calculate_insatisfaction(self):

    if len(self.solicited_subjects)==0:
      return 0.0

    assigned_subject_codes={subject.code for subject in self.assigned_subjects}
    maj_size=len(self.assigned_subjects)
    msj_size=len(self.solicited_subjects)
    if msj_size==0:
      return 0.0
    unsupplied_subjects=[solicited for solicited in self.solicited_subjects if solicited.code not in assigned_subject_codes ]
    sum_pjl=sum(solicited_subject.priority for solicited_subject in unsupplied_subjects)
    fun_Y= msj_size*3-1

    return (1-(maj_size/msj_size))*(sum_pjl/fun_Y)


  def assign_subject(self,subject):

    solicited_subject=next((solicited_subject for solicited_subject in self.solicited_subjects if solicited_subject.code==subject.code),None)
    if solicited_subject:
      self.assigned_subjects.append(solicited_subject)


class SolicitedSubject:
  def __init__(self,subject,priority):
    self.subject=subject
    self.priority:int=priority


  def return_info(self):
    return f'({self.subject.code},{self.priority})'

  @property
  def code(self) -> str:
    return self.subject.code

In [2]:
from typing import List
#implement an example to test
subjects=[
  Subject('M1',3),
  Subject('M2',4),
  Subject('M3',2)
]

students=[
    Student('e1',[SolicitedSubject(subjects[0],5),
                  SolicitedSubject(subjects[1],2),
                  SolicitedSubject(subjects[2],1)
                  ],1),
    Student('e2',[SolicitedSubject(subjects[0],4),
                  SolicitedSubject(subjects[1],1),
                  SolicitedSubject(subjects[2],3)
                  ],2),
    Student('e3',[SolicitedSubject(subjects[1],3),
                  SolicitedSubject(subjects[2],2)
                  ],3),
    Student('e4',[SolicitedSubject(subjects[0],2),
                  SolicitedSubject(subjects[2],3)
                  ],4),
    Student('e5',[SolicitedSubject(subjects[0],3),
                  SolicitedSubject(subjects[1],2),
                  SolicitedSubject(subjects[2],3)
                  ],5)

]

In [3]:
from re import sub
import copy
import math
#brute force algorithm
def combinations(elements, k):


    # Caso base: combinación vacía
    if k == 0:
        return [[]]
    # Caso base: no hay más elementos
    if len(elements) < k:
        return []

    first = elements[0]
    remain = elements[1:]


    # 1. Tomamos al primero y buscamos combinaciones de tamaño k-1 en el resto
    whit_first = [[first] + c for c in combinations(remain, k-1)]

    # 2. No tomamos al primero y buscamos combinaciones de tamaño k en el resto
    whitout_first = combinations(remain, k)

    # Unimos ambos resultados
    return whit_first + whitout_first


def combinations_by_subject(subject,students):
  quotes=subject.quotas
  code=subject.code
  applicant_students=[student for student in students if any([solicitedsubject.code==code for solicitedsubject in student.solicited_subjects])]

  comb =combinations([student.code for student in applicant_students],quotes)

  return comb

def distribution_final(distributions):

  if len(distributions)==0:
    return [[]]

  first_array=distributions[0]

  rest_array=distributions[1:]

  rest_combines=distribution_final(rest_array)

  result=[]
  for element in first_array:
    for comb in rest_combines:
      result.append([element]+comb)

  return result



subject_distribution=[]

for subject in subjects:
  posible_comb=combinations_by_subject(subject,students)
  subject_distribution.append(posible_comb)


distributed_solutions=distribution_final(subject_distribution)

print(distributed_solutions)

# for d in distributed_solutions:
#   print('dist_sol-----------------')
#   for subject_dist in d:
#     print('subject_dist-----------')

#     for student in subject_dist:
#       print(student.code)


final_solutions=[]
for solution in distributed_solutions:
  new_students=copy.deepcopy(students)
  for i in range(len(solution)):
    for student in new_students:
      subject=subjects[i]
      if student.code in solution[i]:
        student.assign_subject(subject)
      # isapplicant=any(solicited_subject.code==subject_code for solicited_subject in student.solicited_subjects )
      # if isapplicant and student.code not in solution[i]:
      #   student.delete_solicited_subject(subject_code)
  final_solutions.append(new_students)




def find_optimal_solution(final_solutions):
  min_general_satisfaction={'min':math.inf,'solution':None}
  for solution in final_solutions:
    acumulated_insatisfaction=0
    # print('---------------------------------------------')
    for i in range(len(solution)):

      student=solution[i]

      student_insatisfaction=student.calculate_insatisfaction()

      acumulated_insatisfaction+=student_insatisfaction

    general_insatisfaction=acumulated_insatisfaction/len(students)
    if min_general_satisfaction['min']>general_insatisfaction:
      min_general_satisfaction['min']=general_insatisfaction
      min_general_satisfaction['solution']=solution

  return min_general_satisfaction


sol=find_optimal_solution(final_solutions)
print(sol['min'])
for student in sol['solution']:
  print(student.return_info_sol())
#----------------------------------------------------------------------depuracion---------------------------------------------------
# for solution in final_solutions:
#   print('-----------------------------------------')

#   for student in solution:
#     print(student.return_info())

# solution=final_solutions[15]

# print(distributed_solutions[15])

# cum=0
# for i in range(len(solution)):
#   res=solution[i].calculate_insatisfaction(students[i].solicited_subjects)
#   print(f'{solution[i].code}:',res)
#   cum+=res

# print(cum/5)


































[[['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e1', 'e2']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e1', 'e3']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e1', 'e4']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e1', 'e5']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e2', 'e3']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e2', 'e4']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e2', 'e5']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e3', 'e4']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e3', 'e5']], [['e1', 'e2', 'e4'], ['e1', 'e2', 'e3', 'e5'], ['e4', 'e5']], [['e1', 'e2', 'e5'], ['e1', 'e2', 'e3', 'e5'], ['e1', 'e2']], [['e1', 'e2', 'e5'], ['e1', 'e2', 'e3', 'e5'], ['e1', 'e3']], [['e1', 'e2', 'e5'], ['e1', 'e2', 'e3', 'e5'], ['e1', 'e4']], [['e1', 'e2', 'e5'], ['e1', 'e2', 'e3', 'e5'], ['e1', 'e5']], [['e1', 'e2', 'e5'], ['e1', 'e2', 'e3', 'e5'], ['e2', 'e3']], [['e1', 'e2', 'e5'], ['e1', 'e2', 'e3', 'e5'], ['e2', 'e4']], [['e1',

In [ ]:
array=[[1]]

w=array+[[]]
w


[[1], []]

In [ ]:
#backtraking algorithm

def find_all_combinations(n: int, k: int):
  """
  Finds all possible ways to choose k positions from n spots.
  """
  results_indices = []

  def backtrack(start_index, current_combo):
      # 1. Base Case: The combination has the right number of "1"s
      if len(current_combo) == k:
          results_indices.append(list(current_combo)) # Add a copy
          return

      # 2. Recursive Step
      for i in range(start_index, n):
          # Choose: Add the current position index
          current_combo.append(i)

          # Explore: Look for the next position from i + 1
          backtrack(i + 1, current_combo)

          # Unchoose (Backtrack): Remove the index to try the next one
          current_combo.pop()

  # Start the recursion from index 0 with an empty combo
  backtrack(0, [])

  # --- Convert indices to the final array format ---
  final_solutions = []
  for indices in results_indices:
      # Create an array of all zeros
      template = [0] * n
      # Place "1"s at the found indices
      for index in indices:
          template[index] = 1
      final_solutions.append(template)

  return final_solutions